In [ ]:
import os
from os.path import basename
import pandas as pd
from pandas import *
import time

In [ ]:
#  Read in the csv files for Parcel ID and Taz and Parcel ID and Use Type for join
parcels = pd.read_csv(r'D:\soundcast\soundcast\inputs\buffered_parcels.dat', sep=' ')

In [ ]:
# Ideally, we attach a land use code to the urbansim input and filter directly from there
# For now, we need to join in parcel land use info
parcel_lu = pd.read_csv(r'R:\Craig\Trucks\inputs\TripGen\Base\parcels\parcels_allowable_lu.txt')

In [ ]:
df = parcels.merge(parcel_lu, left_on='parcelid',right_on='parcel_id')

In [ ]:
# List of allowable truck land uses
truck_uses = ['Agriculture','Fisheries','Forest, harvestable','Forest, protected','Industrial','Military','Mining','Warehousing']

In [ ]:
# select truck rows only, using the allowable truck land uses
truck_df = df[df["generic_land_use_1"].isin(truck_uses)]

In [ ]:
# For future reference, lookup all the use_codes for these generic_land_uses
# return list of tuples that includes the generic name and all the use codes
truck_use_codes = np.array(truck_df.groupby(['generic_land_use_1','use_code']).count().index)    # returns list of tuples
truck_use_codes = [i[1] for i in truck_use_codes]    # extract only the use_code field, drop the generic_land_use_1 data

# in future, probably want to use the use_code field to select truck rows like this:
# df[df['use_code'].isin(truck_use_codes)]

In [ ]:
# Add a flag for truck allowable field
truck_df['trucks_allowed_parcel'] = 1

In [ ]:
# merge the truck_df back into the main df
df = df.merge(truck_df[['parcelid','trucks_allowed_parcel']], how='left')
df['trucks_allowed_parcel'].fillna(0,inplace=True)    # Truck restricted parcels get a 0

In [ ]:
# Now, groupby TAZ and create new flag that allows trucks on TAZ with allowable land use
df_taz = pd.DataFrame(df.groupby('taz_p').sum()[['trucks_allowed_parcel']])
df_taz['trucks_allowed_taz'] = pd.cut(df_taz['trucks_allowed_parcel'], bins=[0,1,df_taz['trucks_allowed_parcel'].max()], labels=[0,1], include_lowest=True)

In [ ]:
df_taz['trucks_allowed_taz'].astype('int').sum()

In [6]:
import h5py
import pandas as pd

In [88]:
# look at the output for midday

newtruck = h5py.File(r'D:\soundcast\soundcast\inputs\4k\auto.h5')

In [89]:
oldtruck = h5py.File(r'R:\SoundCast\releases\TransportationFutures2010\inputs\4k\auto.h5')

In [93]:
# Get results by regional growth center
taz_lookup = pd.read_csv(r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize\TAZ_TAD_County.csv')

In [375]:
h5file = newtruck

In [376]:
production_total = {}
for trk in ['lttrk', 'mdtrk','hvtrk']:
    df = 0
    for tod in ['am','md','pm','ev','ni']:
        df += pd.DataFrame(h5file[tod][tod[0]+trk][:])
    df = pd.DataFrame([df.sum(axis=0),df.index]).T
    df.columns = [trk,'TAZ']
    df['TAZ'] = [i+1 for i in df['TAZ']]    # offset TAZ by 1 to get true value
    
    # Join to geographic data
    df = df.merge(taz_lookup, on='TAZ', how='left')
    
    production_total[trk] = df

In [377]:
attraction_total = {}
for trk in ['lttrk', 'mdtrk','hvtrk']:
    df = 0
    for tod in ['am','md','pm','ev','ni']:
        df += pd.DataFrame(h5file[tod][tod[0]+trk][:])
    df = pd.DataFrame([df.sum(axis=1),df.index]).T
    df.columns = [trk,'TAZ']
    df['TAZ'] = [i+1 for i in df['TAZ']]    # offset TAZ by 1 to get true value
    
    # Join to geographic data
    df = df.merge(taz_lookup, on='TAZ', how='left')
    
    attraction_total[trk] = df

In [378]:
# Format output
pd.options.display.float_format = '{:,.0f}'.format

In [379]:
truck_list = ['lttrk', 'mdtrk','hvtrk']

In [431]:
attr = pd.DataFrame([attraction_total[trk].groupby('County').sum()[trk] for trk in truck_list])
attr.loc['Total'] = attr.sum()
attr

County,King,Kitsap,Pierce,Snohomish
lttrk,"188,652","17,072","58,244","53,199"
mdtrk,"122,746","11,092","38,309","35,216"
hvtrk,"96,846","6,056","28,730","23,453"
Total,"408,243","34,220","125,284","111,868"


In [432]:
prod = pd.DataFrame([production_total[trk].groupby('County').sum()[trk] for trk in truck_list])
prod.loc['Total'] = prod.sum()
prod

County,King,Kitsap,Pierce,Snohomish
lttrk,"188,652","17,072","58,244","53,199"
mdtrk,"122,746","11,092","38,309","35,216"
hvtrk,"97,299","6,389","28,710","24,216"
Total,"408,696","34,553","125,264","112,631"


In [505]:
########
# Join prod and attr
df = pd.concat([prod,attr], axis=1)[prod.columns]    # join dataframes and sort w/ county names adjacent
# df = df
df.columns = pd.MultiIndex.from_product([np.array(prod.columns),['Productions','Attractions']])
# df.to_clipboard()
df

King                  Kitsap                  Pierce              \
      Productions Attractions Productions Attractions Productions Attractions   
lttrk     188,652     188,652      17,072      17,072      58,244      58,244   
mdtrk     122,746     122,746      11,092      11,092      38,309      38,309   
hvtrk      97,299      96,846       6,389       6,056      28,710      28,730   
Total     408,696     408,243      34,553      34,220     125,264     125,284   

        Snohomish              
      Productions Attractions  
lttrk      53,199      53,199  
mdtrk      35,216      35,216  
hvtrk      24,216      23,453  
Total     112,631     111,868

In [506]:
attr = pd.DataFrame([attraction_total[trk].groupby('New DistrictName').sum()[trk] for trk in truck_list])
attr.loc['Total'] = attr.sum()
attr

New DistrictName,East Side,Everett-Lynwood-Edmonds,Kitsap,North Seattle-Shoreline,Renton-FedWay-Kent,S.Kitsap,Seattle CBD,South Pierce,Suburban Snohomish,Tacoma,West-South Seattle
lttrk,"59,475","29,787","17,072","23,868","52,429","3,485","30,763","36,093","23,412","18,666","22,115"
mdtrk,"39,592","19,631","11,092","15,701","33,796","2,285","19,508","24,017","15,585","12,007","14,148"
hvtrk,"25,273","14,493","6,056","7,925","32,050","1,133","13,021","15,452","8,960","12,145","18,577"
Total,"124,340","63,911","34,220","47,494","118,276","6,903","63,292","75,562","47,957","42,818","54,840"


In [507]:
prod = pd.DataFrame([production_total[trk].groupby('New DistrictName').sum()[trk] for trk in truck_list])
prod.loc['Total'] = prod.sum()
prod

New DistrictName,East Side,Everett-Lynwood-Edmonds,Kitsap,North Seattle-Shoreline,Renton-FedWay-Kent,S.Kitsap,Seattle CBD,South Pierce,Suburban Snohomish,Tacoma,West-South Seattle
lttrk,"59,475","29,787","17,072","23,868","52,429","3,485","30,763","36,093","23,412","18,666","22,115"
mdtrk,"39,592","19,631","11,092","15,701","33,796","2,285","19,508","24,017","15,585","12,007","14,148"
hvtrk,"25,125","14,983","6,389","8,147","31,321","1,132","13,954","15,455","9,233","12,123","18,753"
Total,"124,192","64,401","34,553","47,716","117,547","6,902","64,226","75,565","48,230","42,797","55,016"


In [429]:
prod.columns

Index([u'East Side', u'Everett-Lynwood-Edmonds', u'Kitsap', u'North Seattle-Shoreline', u'Renton-FedWay-Kent', u'S.Kitsap', u'Seattle CBD', u'South Pierce', u'Suburban Snohomish', u'Tacoma', u'West-South Seattle'], dtype='object')

In [508]:
# Join prod and attr
df = pd.concat([prod,attr], axis=1)[prod.columns]    # join dataframes and sort w/ county names adjacent
# df = df
df.columns = pd.MultiIndex.from_product([np.array(prod.columns),['Productions','Attractions']])
# df.to_clipboard()
df

East Side             Everett-Lynwood-Edmonds                  Kitsap  \
      Productions Attractions             Productions Attractions Productions   
lttrk      59,475      59,475                  29,787      29,787      17,072   
mdtrk      39,592      39,592                  19,631      19,631      11,092   
hvtrk      25,125      25,273                  14,983      14,493       6,389   
Total     124,192     124,340                  64,401      63,911      34,553   

                  North Seattle-Shoreline             Renton-FedWay-Kent  \
      Attractions             Productions Attractions        Productions   
lttrk      17,072                  23,868      23,868             52,429   
mdtrk      11,092                  15,701      15,701             33,796   
hvtrk       6,056                   8,147       7,925             31,321   
Total      34,220                  47,716      47,494            117,547   

                      ...     Seattle CBD             South Pierce  \
      Attractions     ...     Productions Attractions  Productions   
lttrk      52,429     ...          30,763      30,763       36,093   
mdtrk      33,796     ...          19,508      19,508       24,017   
hvtrk      32,050     ...          13,954      13,021       15,455   
Total     118,276     ...          64,226      63,292       75,565   

                  Suburban Snohomish                  Tacoma              \
      Attractions        Productions Attractions Productions Attractions   
lttrk      36,093             23,412      23,412      18,666      18,666   
mdtrk      24,017             15,585      15,585      12,007      12,007   
hvtrk      15,452              9,233       8,960      12,123      12,145   
Total      75,562             48,230      47,957      42,797      42,818   

      West-South Seattle              
             Productions Attractions  
lttrk             22,115      22,115  
mdtrk             14,148      14,148  
hvtrk             18,753      18,577  
Total             55,016      54,840  

[4 rows x 22 columns]